In [ ]:
import tensorflow as tf


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config= ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction=0.5
session=InteractiveSession(config=config)

In [ ]:
#import all needful librarries
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
from tensorflow.keras.applications.resnet50 import  ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input


In [ ]:
resnet=ResNet50(input_shape=image_size+[3], weights="imagenet",include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
#resize all images to this
image_size=[224,224]
trai_path= '/content/train'
valid_path= '/content/test'

In [ ]:
#dont traiin exisring weights
for layer in resnet.layers:
  layer.trainable=False

In [ ]:
#useful for getting number of output classes
folders=glob('/content/train')

In [ ]:
x=Flatten()(resnet.output)

In [ ]:
len(folders)

1

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)


In [ ]:
#creating model
model=Model(inputs=resnet.inputs,outputs=prediction)

In [ ]:
#view structure of the model
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
training_set=train_datagen.flow_from_directory('/content/train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 1951 images belonging to 4 classes.


In [ ]:
test_set=train_datagen.flow_from_directory('/content/test',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical')

Found 36 images belonging to 5 classes.


In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

In [ ]:
#using ResNet50V2